## Big Data Assignment: Liquor Sales in Iowa
Giana Grace and Natalie Dume

**Introduction**

This dataset provides a detailed overview of liquor sales in Iowa, encompassing a wide range of data collected between 2012 and 2018 from multiple cities and stores. The dataset includes essential information such as store addresses, transaction counts, the number of bottles sold, bottle costs, and associated sales in dollars. It also contains specific details on product categories, bottle volumes, and pricing for both retailers and consumers. By analyzing trends across the dataset, key insights into Iowa's liquor consumption patterns, regional sales performance, and the economic impact of alcohol sales on the local economy can be uncovered.

Liquor sales in Iowa are primarily driven by two factors: volume sold and price point. Higher sales volumes, particularly in terms of bottles and liters sold, will have a direct correlation with higher overall sales revenue. In addition, stores that offer more affordable retail pricing per bottle will experience higher sales due to price-sensitive consumer behavior. Regional differences, such as higher sales in urban areas compared to rural regions, may also reflect varying consumer preferences and disposable income levels.

To gain deeper insights and make more accurate predictions regarding liquor sales, a Random Forest model was employed. This approach is well-suited for identifying important factors that influence total sales, such as vendor sources, store locations, and state retail prices.We also reduced the dimensions of the data to allow for more efficient processing, and used feature selection, sampling, and removing features with low importance. In addition to this, we simplified the data and targeted features. The model helps in understanding how different features contribute to sales variability, uncovering key drivers of revenue in Iowa's liquor market.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import seaborn as sns
import psutil
sns.set(style="darkgrid")

In [ ]:
!python -m pip install dask[dataframe] --upgrade

**Processing the Data**

In [ ]:
import dask.dataframe as dd

df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Liquor_Sales.csv", nrows=20000)

df.head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S24127700024,02/19/2015,3678,"Smoke Shop, The",1918 SE 14TH ST,DES MOINES,50320,POINT (-93.597011 41.570844),77.0,Polk,...,41783,Uv Blue Raspberry Vodka Mini,6,500,4.89,7.34,2,14.68,1.0,0.26
1,S15066200002,10/10/2013,2633,Hy-Vee #3 / BDI / Des Moines,3221 SE 14TH ST,DES MOINES,50320,POINT (-93.596754 41.554101),77.0,Polk,...,904969,Sabe Premiom Sake Double Barrel,6,750,14.99,22.49,6,134.94,4.5,1.19
2,S19323500030,06/03/2014,2607,Hy-Vee Wine and Spirits / Shenandoah,520 SO FREMONT,SHENANDOAH,51601,POINT (-95.385111 40.761736),73.0,Page,...,45277,Paramount White Rum,12,1000,4.34,6.51,12,78.12,12.0,3.17
3,S23334500013,01/06/2015,4810,Kum & Go #518 / Ankeny,3603 NE OTTERVIEW CIRCLE,ANKENY,50021,POINT (-93.572458 41.760989),77.0,Polk,...,43121,Bacardi Superior Rum Mini,12,500,5.54,8.31,1,8.31,0.5,0.13
4,S09742200010,12/27/2012,4025,Karam Kaur Khasriya Llc,702 13TH ST,BELLE PLAINE,52208,POINT (-92.277759 41.897052),6.0,Benton,...,11298,Crown Royal Canadian Whisky,6,1750,31.00,46.49,2,92.98,3.5,0.92


In [ ]:
print(df.dtypes)


Invoice/Item Number       object
Date                      object
Store Number               int64
Store Name                object
Address                   object
City                      object
Zip Code                  object
Store Location            object
County Number            float64
County                    object
Category                 float64
Category Name             object
Vendor Number              int64
Vendor Name               object
Item Number                int64
Item Description          object
Pack                       int64
Bottle Volume (ml)         int64
State Bottle Cost        float64
State Bottle Retail      float64
Bottles Sold               int64
Sale (Dollars)           float64
Volume Sold (Liters)     float64
Volume Sold (Gallons)    float64
dtype: object


After importing the necessary libraries, we conduct the formation of the random forests. Preprocessing continues with the handling of missing data, defining the target value and dropping unnecessary columns to reveal a clean and workable dataset. In addition to doing so, the features (x-value), and target (y-value), which is liquor sales, is the variable the model will predict. This will aid in prediction. Feature importance was conducted to see which variables in a dataset have the most influence on the model's predictions, as seen the most important feature was Volume Sold (Liters) and Bottles Sold.

The training will reveal patterns and evaluate feature importance showing the key drivers behind liquor sales predictions.

Categorical data is converted into numerical format to help with interpretation by use of LabelEncoder.

Gradient Boosting helps with fitting the model by. learning the relationship between the input features and the target. Following this, the feature importance gives a score for each feature to see which features have the biggest impact -- a dataframe visualizes and aids in sorting in importance.



In [ ]:

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
import seaborn as sns



In [ ]:
pip install scikit-learn

**Feature Importance**

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor


df = df.dropna()
target = 'Sale (Dollars)'
X = df.drop(columns=[target, 'Item Description', 'Date', 'Item Number', 'Category', 'Store Location', 'Invoice/Item Number', 'City', 'Bottle Volume (ml)', 'Store Name', 'Vendor Name', 'Vendor Number', 'Category Name', 'Store Number', 'Address', 'Zip Code', 'County', 'Pack', 'County Number' ])  # Features
y = df[target]  # Target variable

le = LabelEncoder()
X = X.apply(le.fit_transform)


# Train a Gradient Boosting model
model = GradientBoostingRegressor()
model.fit(X, y)

# Find feature importances
importances = model.feature_importances_
feature_names = X.columns

# create a dataframe for the feature importances
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print(importance_df)

                 Feature  Importance
4  Volume Sold (Gallons)    0.383534
2           Bottles Sold    0.339545
3   Volume Sold (Liters)    0.172786
0      State Bottle Cost    0.058755
1    State Bottle Retail    0.045380


In [ ]:
features = pd.get_dummies(df)
features.head(5)

labels = np.array(features['Sale (Dollars)'])

# Drop the target variable from the features
features= features.drop('Sale (Dollars)', axis = 1)

# Saving feature names for later use
feature_list = list(features.columns)

features = np.array(features)

**Random Forest**

Next, we split the data into training and testing sets and add a Random Forest Classifier for classification and regression tasks. The data split of 80% and 20% ensures the model is trained on one portion of the data and tested on a separate portion to evaluate its performance.

Additionally, to using the Random Forest Regressor, 100 decision trees are trained to ensure consistent results for multiple runs.

There's a R^2 squared value which is appropiate for predicting continuous outcomes.
The goal is to understand the proportion of variance explained by the model. In this case, we are predicting sales revenue for liquor. We end up with a R^2 value of 0.82923325293366, meaning 80% of the variance in the liquor sales (target) variable can be explained by the independent (feature) variables.

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import r2_score


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Chose 4 max depth to be able to see visualization
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=4)


rf = rf_reg.fit(X_train, y_train)


y_pred = rf_reg.predict(X_test)
r = r2_score(y_test, y_pred)

print(f"r^2 score: {r}")


r^2 score: 0.82923325293366


In [ ]:
from sklearn.tree import export_graphviz
import pydot


tree = rf.estimators_[5]


export_graphviz(tree, out_file = 'tree.dot', rounded = True, precision = 1)


(graph, ) = pydot.graph_from_dot_file('tree.dot')


graph.write_png('tree.png');

**Conclusion**

After analyzing the liquor sales dataset using a Random Forest model, we can draw important conclusions regarding the relationship between key features and sales outcomes. The most influential factors impacting liquor sales are ‘Volume Sold (Liters and Gallons)’ and ‘Bottles Sold’, as they directly reflect the quantity and size of liquor purchases. Additionally, pricing-related variables, such as ‘State Bottle Cost’ and ‘State Bottle Retail’, play a critical role in determining revenue, albeit with a lesser impact than the volume-based features.

In the decision tree diagram, the model aims to minimize the squared error when predicting 'Sale (Dollars)'. The root node splits based on ‘Volume Sold (Liters)’, the most significant feature with an importance score of 0.384. At this first split, data is divided based on whether the volume sold is less than or equal to 101.5 liters, with an average sales value of 123.80 (dollars) and a squared error of 119868.3. Further splits occur based on ‘Bottles Sold’ (importance score of 0.339) at x[4] <= 56.5, predicting sales at $115. This helps reduce the squared error to 44204.4, showing how volume and bottles sold dominate the tree's early splits to drive the prediction of sales values.

As the tree progresses, splits based on ‘State Bottle Retail’ (importance score 0.038) further refine the sales predictions, leading to nodes that predict high sales values, such as 1980.60(dollars). Additional splits involving features like ‘Volume Sold (Gallons)’ (importance score of 0.173) and further price-based splits result in terminal nodes that predict final sales values ranging from 446.80 to 2564.10 (dollars). The right subtree similarly uses ‘Volume Sold (Gallons)’ and price features like ‘State Bottle Cost’ to continue refining predictions, with one terminal node accurately predicting sales of $31344.0 with zero error for a small subset of the data.

The structure of the decision tree reveals that features with the highest importance— ‘Volume Sold’ and ‘Bottles Sold’ —dominate the upper nodes, as they offer the most significant reductions in error. In contrast, pricing features like ‘State Bottle Cost’ and ‘State Bottle Retail’, while relevant, appear lower in the tree and provide more granular adjustments to the model's predictions. This hierarchy reflects how the model balances both volume and pricing to optimize predictions for 'Sale (Dollars)'.

In real-world liquor sales trends, the relationship between quantity sold and pricing is often a key driver of revenue. Larger volumes, particularly for high-demand items, contribute more to total sales, while competitive pricing strategies can further influence consumer purchasing behavior. The decision tree helps illustrate how these factors interact to predict sales outcomes effectively.

Overall, the Random Forest model demonstrates the importance of these features by segmenting the data based on volume and pricing characteristics, while also improving accuracy and reducing overfitting. This analysis provides valuable insights into liquor sales, potentially informing pricing strategies, inventory management, and sales forecasting for businesses in the industry.

In [ ]:
f = files.upload()

# Convert ipynb to html
import subprocess
file0 = list(f.keys())[0]
_ = subprocess.run(["pip", "install", "nbconvert"])
_ = subprocess.run(["jupyter", "nbconvert", file0, "--to", "html"])

# download the html
files.download(file0[:-5]+"html")

Saving Big_Sales_Data (2).ipynb to Big_Sales_Data (2).ipynb


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>